# Label Roads

For machine learning, a set of road labels are needed for the downloaded aerial images. That is, for each aerial image, a mask image the same size is needed with each pixel having value 1 or 0 to indicate the prescense or abscense of a road. 


<table><tr><td><img src='/img/notebook/label_example_img.png'></td><td><img src='/img/notebook/label_example_label.png'></td></tr></table>


Here, we use Open Street Map (OSM) data to create binary road masks for the aerial images as shown above. The OSM data is in the form of lines denoted by sequences of geographic coordinates, and the aerial images are georeferenced meaning each pixel can be mapped to a coordinate pair. Thus, assigning labels is relaively straightforward by mapping the road coordinates to the pixels in the images. There are two notable shortcomings of this approach:

1. OSM data may sometimes be incomplete or inaccurate.
2. OSM gives only the location of the center of the road and not the full extend of the road width. 

The first issue is hard to correct, but with enough data a neural net can hopefully overcome the noise.

The second issue can be approached by assigning road labels more liberally. Rather than only assigning the centerline pixel as a road, one can label the adjacent neighboring pixels as roads as well. Methodical refinements of this procedure include expanding the neighborhood based on road type (e.g. highways have a larger neighborhood than residential streets) or by assigning a probability distribution to neighboring pixels rather than hard 1's. However, for this project, it is sufficient simply to expand the road labels by a fixed amount (this has already been applied in the example above). Compare the undilate (left) and dilated label examples below.

<table><tr><td><img src='/img/web/labels_no_dilation.png'></td><td><img src='/img/web/labels_dilation.png'></td></tr></table>

In this rest of this notebook, a label image (i.e. a binary mask) is generated for each NAIP image downloaded previously. These images are of course the same size as the NAIP image and stored locally. Then, for the large city (Phoenix, AZ) which serves as the training and benchmark set, each image/mask pair is broken up into smaller tiles (say, 512x512x3 pixels) that will be fed as input to a neural net. These tilings are saved as datasets in the hdf5 format.

In [1]:
import rasterio
import fiona
import json
import h5py
import cv2
import os

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from rasterio.features import rasterize 
from helpers import make_tiles
from pyproj import Proj
from PIL import Image

%matplotlib inline

/home/carlos/anaconda3/envs/geo/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First, we need to figure out which coordinate reference system (CRS) / projections we're working with. Different images may have different projections depending on their location, so the road coordinates need to be mapped with the correct projection. 

It's a little overkill, but here we simply project all roads in Arizona for each CRS we find. If memory were a constrained resource, we could limit it to only roads within the cities that were downloaded, but the projections for a single state are managable.

In [8]:
from importlib import reload
import helpers
reload(helpers)
from helpers import make_tiles

In [2]:
with open('data/naip/download_info.json', 'r') as places_in:
    places = json.load(places_in)

## Get all GeoTiff paths as a flat list
tif_paths_in = [place_info['img_paths'] for _, place_info in places.items()]
tif_paths_in = [path_in for paths_in in tif_paths_in for path_in in paths_in]

## Get projections
projections = []
for tif_path_in in tif_paths_in:
    with rasterio.open(tif_path_in) as tif_in:
        projections.append(tif_in.crs['init'])
projections = list(set(projections))

print(projections)    

['epsg:26911', 'epsg:26912']


In [3]:
## Getting shapes for all roads in AZ
shape_path = 'data/osm/arizona-latest-free_shp/gis.osm_roads_free_1.shp'
roads_map = {} # Key is projection CRS, value is list of projected roads
for projection in projections:

    ## Get transformation
    proj = Proj(init = projection)
    
    ## Project road coordinates
    roads = []
    for i, feat in enumerate(fiona.open(shape_path, 'r')):
        lons, lats = zip(*feat['geometry']['coordinates'])
        xx, yy = proj(lons, lats)
        road = {'type': 'LineString','coordinates': list(zip(xx,yy))} # In meters
        roads.append(road)
    roads_map[projection] = roads

print('Found {} roads'.format(len(roads_map[projections[0]])))

Found 531899 roads


Next, loop through each image, get its CRS, and overlay the roads with the corresponding projection. A dilation from the OpenCV library is used to expand road labels.

In [4]:
## Save labels as .PNG images
## Writing roads within bounds of a source geotiff.
labels_dir = 'data/naip/img/labels/'
kernel = np.ones((3,3), np.uint8) # For label dilation

## Make one output label per input image
for tif_path_in in tif_paths_in:
    labels_name_out = tif_path_in.split('/')[-1].replace('.tif', '_labels.png')
    labels_path_out = labels_dir + labels_name_out
    
    ## Skip if we've already made it
    if os.path.isfile(labels_path_out):
        continue
    
    with rasterio.open(tif_path_in) as tif_in:
        roads = roads_map[tif_in.crs['init']]

        ## Rasterize a mask
        labels = rasterize( 
            roads, 
            out_shape = tif_in.shape, 
            transform = tif_in.transform,
            default_value = 1,
            fill = 0,
            all_touched=True
        )
        labels = cv2.dilate(labels, kernel, iterations = 2)
    
        labels_img = Image.fromarray(labels * 255)
        labels_img.save(labels_path_out)

The data from Phoenix is used as the train/test/dev sets and will be stored in a hdf5 file. Two helper functions will accomplish this. First, `make_tiles` takes an image and chunks it up into smaller sizes that can be input to the neural net. Further, we can specify if there should be any padding which there should be for the input image because the neural net reduces the size of the input. In this case, the padding comes from reflecting the edges of the input. We tile both the aerial image and the corresponding label image. The code is in `helpers.py`.

Then, `make_hdf5_set` defined below takes a list of multiple aerial/label image pairs, splits each into tiles (called chunks in the code), and randomly assigns the tiles to the train/dev/test sets in specified proportions.

In [9]:
def make_hdf5_set(
    hdf5_path,
    img_paths,
    frac_train = .80,
    frac_dev = .10,
    frac_test = .10,
    train_input_name = 'X_train',
    train_label_name = 'Y_train',
    dev_input_name = 'X_dev',
    dev_label_name = 'Y_dev',
    test_input_name = 'X_test',
    test_label_name = 'Y_test'
):
    assert frac_train + frac_dev + frac_test == 1
    
    with h5py.File(hdf5_path, 'w') as data:

        chunk_counter = 0
        for i,img_path in enumerate(img_paths):

            ## Chunk the image and corresponding labels
            labels_path = img_path.replace('download', 'labels').replace('.tif', '_labels.png')
            X_chunks, _, _ = make_tiles(img_path, pad = 64)
            labels_chunks, _, _ = make_tiles(labels_path)
            labels_chunks = labels_chunks / labels_chunks.max()
            labels_chunks = np.expand_dims(labels_chunks, 3).astype(np.int8)
            chunk_counter = chunk_counter + X_chunks.shape[0]

            ## Split into train/dev/test
            X_train, X_test, Y_train, Y_test = train_test_split(X_chunks, labels_chunks, test_size=frac_test, random_state=40)
            X_train, X_dev, Y_train, Y_dev = train_test_split(X_train, Y_train, train_size=frac_train/(frac_train+frac_dev), random_state=30)

            ## Add first chunks to dataset
            ## Should make the maxshape not so hardcoded
            if i == 0:
                dset_x_train = data.create_dataset(train_input_name, X_train.shape, maxshape = (None, 640, 640, 3), data=X_train)
                dset_x_dev = data.create_dataset(dev_input_name, X_dev.shape, maxshape = (None, 640, 640, 3), data=X_dev)
                dset_x_test = data.create_dataset(test_input_name, X_test.shape, maxshape = (None, 640, 640, 3), data=X_test)
                dset_y_train = data.create_dataset(train_label_name, Y_train.shape, maxshape = (None, 512, 512, 3), data=Y_train)
                dset_y_dev = data.create_dataset(dev_label_name, Y_dev.shape, maxshape = (None, 512, 512, 3), data=Y_dev)
                dset_y_test = data.create_dataset(test_label_name, Y_test.shape, maxshape = (None, 512, 512, 3), data=Y_test)  

            ## Append new chunks to the dataset
            else:
                n_train_before_resize = dset_x_train.shape[0]
                n_train_after_resize = n_train_before_resize + X_train.shape[0]
                n_dev_before_resize = dset_x_dev.shape[0]
                n_dev_after_resize = n_dev_before_resize + X_dev.shape[0]
                n_test_before_resize = dset_x_test.shape[0]
                n_test_after_resize = n_test_before_resize + X_test.shape[0]

                dset_x_train.resize(n_train_after_resize, axis = 0)
                dset_y_train.resize(n_train_after_resize, axis = 0)
                dset_x_dev.resize(n_dev_after_resize, axis = 0)
                dset_y_dev.resize(n_dev_after_resize, axis = 0)
                dset_x_test.resize(n_test_after_resize, axis = 0)
                dset_y_test.resize(n_test_after_resize, axis = 0)

                dset_x_train[n_train_before_resize:] = X_train
                dset_y_train[n_train_before_resize:] = Y_train
                dset_x_dev[n_dev_before_resize:] = X_dev
                dset_y_dev[n_dev_before_resize:] = Y_dev
                dset_x_test[n_test_before_resize:] = X_test
                dset_y_test[n_test_before_resize:] = Y_test
            
        print('Saved {} input/output pairs to {}'.format(chunk_counter, hdf5_path))


Since the whole Phoenix dataset is rather large (~25GB HDF5 file), for development purposes we'll create a smaller set based on only a few input tiles that we manually specify. Then we'll do the same for the whole dataset.

In [20]:
img_paths = [
    'm_3311117_ne_12_1_20150601',
    'm_3311117_sw_12_1_20150529',
    'm_3311117_nw_12_1_20150529',
    'm_3311117_se_12_1_20150601',
    'm_3311125_ne_12_1_20150601',
    'm_3311125_nw_12_1_20150529',
    'm_3311125_se_12_1_20150601',
    'm_3311125_sw_12_1_20150529',
    'm_3311133_ne_12_1_20150601',
    'm_3311133_nw_12_1_20150529',
    'm_3311133_se_12_1_20150601',
    'm_3311133_sw_12_1_20150529'
]
img_paths = ['data/naip/img/download/' + img_path + '.tif' for img_path in img_paths]
hdf5_path = 'data/naip/hdf5/phoenix_subset.h5'
make_hdf5_set(hdf5_path, img_paths)

/home/carlos/anaconda3/envs/geo/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Saved 2016 input/output pairs to data/naip/hdf5/phoenix_subset.h5


In [10]:
img_paths = places['Phoenix']['img_paths']
hdf5_path = 'data/naip/hdf5/phoenix.h5'
make_hdf5_set(hdf5_path, img_paths)

/home/carlos/anaconda3/envs/geo/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Saved 16128 input/output pairs to data/naip/hdf5/phoenix2.h5
